In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import requests
import urllib
import wget
import os.path
import shutil
from tqdm import tqdm, tqdm_notebook
from bs4 import BeautifulSoup
tqdm.pandas()

In [2]:
db_folder = Path("/media/jakob/bigdata/datasets/archive_org/") 
identifier = "pointlinetoplane00kand"
os.chdir(str(db_folder))

In [38]:
collection = pd.read_csv("guggenheimmuseum.csv")
collection = collection.sort_values('identifier')
collection.head()

,identifier
197,100works00solo
27,19artistsemergen00solo
148,60worksp00pegg
2,acquisitionsof1900reba
64,acuarel00kand


In [96]:
class AOrg():
    download_url = "http://archive.org/download/"
    files_templates = {
        "files": "{id}/{id}_files.xml",
        "meta" : "{id}/{id}_meta.xml",
        "jp2" : "{id}/{id}_jp2.zip"
    }
    def path(identifier, kind='files'):
        return Path(AOrg.files_templates[kind].format(id=identifier))
    
    def url(identifier,kind='files'):
        return AOrg.download_url + str(AOrg.path(identifier, kind))
    
    def fname(identifier, kind='files'):
        return AOrg.path(identifier,kind).name

p = Path(AOrg.path('barnettnewmansta00allo'))
AOrg.url('barnettnewmansta00allo','jp2')
# 
# p.parent.mkdir(exist_ok=True)
# p.exists()
# p.parent / (p.name + '.temp')

'http://archive.org/download/barnettnewmansta00allo/barnettnewmansta00allo_jp2.zip'

In [39]:
identifiers = collection["identifier"]
# collection["files"] = identifiers.progress_apply(AOrg.file)
# collection["jp2"] = identifiers.progress_apply(AOrg.file, kind='jp2')

collection['identifier'][0]

'barnettnewmansta00allo'

In [97]:
def get(url, out, skip_existing=True):
    temp = out.parent / (out.name + '.temp')
    out.parent.mkdir(parents=True, exist_ok=True)
    if out.exists():
        print("skip " + str(out))
    else:
        print("load\t " + str(url))
        print("\t" + str(out))
        try:
            dpath = wget.download(str(url), out=str(temp))
            shutil.move(dpath, str(out))
        except urllib.error.HTTPError:
            print("404: " + str(url))

In [100]:
for doc in tqdm(collection.itertuples(), total=len(collection)):
    for kind in ["files", 'jp2']:
        get(AOrg.url(doc.identifier,kind), AOrg.path(doc.identifier, kind))

  0%|          | 0/204 [00:00<?, ?it/s]

skip 100works00solo/100works00solo_files.xml
load	 http://archive.org/download/100works00solo/100works00solo_jp2.zip
	100works00solo/100works00solo_jp2.zip


KeyboardInterrupt: 

In [ ]:
'https://archive.org/download/100works00solo/100works00solo_jp2.zip'
'http:/archive.org/download/100works00solo/100works00solo_jp2.zip'

In [52]:
# collection.apply(get_file, axis=1)
#     file_name = get_file(db_folder, doc.files)
#     print(file_name)
collection.head()

,identifier
197,100works00solo
27,19artistsemergen00solo
148,60worksp00pegg
2,acquisitionsof1900reba
64,acuarel00kand


In [141]:
def get_file(file):
    if os.path.isfile(str(file)):
        return file
    else:
        return np.nan

In [103]:
def read_attr(name, kind ='files'):
    path_files_xml = AOrg.path(name)
    file_name = AOrg.fname(name,kind)
    
    with path_files_xml.open() as fp:
        files = BeautifulSoup(fp, "lxml")
        attr = files.find_all(name='file', attrs={'name':file_name})
        if len(attr) == 1:
            return {tag.name: tag.text for tag in  attr[0].find_all(True)}
        else:
            return {}


name = 'barnettnewmansta00allo'
attr = read_attr(name,'jp2')

In [104]:
attr

{'crc32': '04646a60',
 'format': 'Single Page Processed JP2 ZIP',
 'md5': 'a59431bb4c02e0137892920add9377ce',
 'mtime': '1305135539',
 'original': 'barnettnewmansta00allo_orig_jp2.tar',
 'sha1': '81956d5c1de245e5038ecc49039d98572592eae0',
 'size': '20140304'}

In [106]:
attrs = collection['identifier'].apply(read_attr, kind='jp2')

In [107]:
collection['size'] = attrs.apply(lambda attr: int(attr.get('size', '0')))

In [108]:
collection['size'].sum() / 1024 / 1024 / 1024

12.646235339343548

In [49]:
files.find(name='file', attrs={'name':file_name})

In [ ]:
shutil.un